TFM FRIENDS

Autora: Mar Burgueño Martín
Máster Big Data & Data Science
Universidad Complutense de Madrid

**ANÁLISIS DESCRIPTIVO DEL CONJUNTO DE DATOS (EDA)**


In [ ]:
#Importación de librerías y herramientas
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
from gensim import corpora
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob
import seaborn as sns


# Descargar recursos de NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')


In [ ]:
from google.colab import files
#Carga el archivo desde tu PC
uploaded = files.upload()

#Obtiene el nombre del archivo cargado
file_name = next(iter(uploaded))

#Lee el archivo CSV en un DataFrame de Pandas
df = pd.read_csv("Friends.csv")


#Mostrar las primeras filas del DataFrame
print("Primeras filas del DataFrame:")
print(df.head())

#Obtener información sobre el DataFrame
print("\nInformación del DataFrame:")
print(df.info())

#Verificar si hay valores nulos
print("\nValores nulos en el DataFrame:")
print(df.isnull().sum())

#Verificar duplicados
print("\nNúmero de duplicados en el DataFrame:", df.duplicated().sum())

#Obtener tamaño del dataset
print("\nTamaño del dataset:")
print(df.shape)


*   Vemos que el conjunto de datos contiene 69.974 filas y cinco columnas: Text, Speaker, Episode, Season y Show. Ya desde el primer momento podríamos descartar la columna Show ya que hace referencia a Friends y no aporta más información.
*   Hay valores nulos en la columna Speaker, con 6.284 valores faltantes. Esto cabe suponer que hay diálogos sin asociar a ningún personaje específico.
* Hay 858 filas duplicadas en el conjunto de datos.


Antes de seguir con el análisis vamos a tratar los valores nulos y los duplicados. Los valores nulos los imputaremos por 'Unknown' y los duplicados los eliminaremos.

In [ ]:
#Imputar valores nulos en la columna "Speaker"
df['Speaker'].fillna('Unknown', inplace=True)

#Verificar que ya no hay valores nulos en la columna "Speaker"
print("Valores nulos en la columna 'Speaker' después de imputar:", df['Speaker'].isnull().sum())

#Eliminar filas duplicadas
df.drop_duplicates(inplace=True)

#Verificar que ya no hay duplicados
print("Número de duplicados en el DataFrame después de eliminarlos:", df.duplicated().sum())

#Obtener el nuevo tamaño del dataset después de eliminar duplicados
print("\nNuevo tamaño del dataset después de eliminar duplicados:", df.shape)



Una vez tratados los valores nulos y duplicados seguimos con el análisis. Visualizamos la distribución de los personajes (Speakers) en los diálogos de Friends.

Antes vamos a agrupar los personajes principales, que son los que nos interesan a la hora de llevar a cabo este proyecto. En una serie como Friends hay  muchísimos personajes secundarios, cameos de una sola vez, etc que no aportan información relevante.

In [ ]:
df['Speaker']

Convertimos todas las cadenas de texto en minúsculas antes de filtrar el df para que aparezcan todas las variantes de nombres. Visualizamos la **Distribución de Personajes Principales en los Diálogos de Friends:**

In [ ]:
#Convertir todos los nombres de personajes a minúsculas
df['Speaker'] = df['Speaker'].str.lower()

#Lista de protagonistas principales en minúsculas
personajes_principales = ['joey', 'monica', 'rachel', 'phoebe', 'ross', 'chandler']

#Filtrar el DataFrame para incluir solo los diálogos de los protagonistas
df_personajes_principales = df[df['Speaker'].isin(personajes_principales)]

#Calcular la frecuencia de cada hablante
speaker_counts_protagonistas = df_personajes_principales['Speaker'].value_counts()

#Graficar la distribución de hablantes de los protagonistas
plt.figure(figsize=(10, 6))
speaker_counts_protagonistas.plot(kind='bar', color='lightcoral')
plt.title('Distribución de Personajes Principales en los Diálogos de Friends')
plt.xlabel('Hablante')
plt.ylabel('Frecuencia')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


En este gráfico se puede apreciar el protagonismo de ciertos personajes frente a otros. Ross es el que más cantidad de texto tiene a lo largo de toda la serie, muy seguido de Rachel y Joey, mientras que Monica y Phoebe son las que más atrás se quedan en comparación con sus compañeros.

**Análisis de las palabras más frecuentes en los diálogos de Friends**

In [ ]:
#Obtener una lista de todas las palabras en los diálogos
all_words = ' '.join(df['Text']).lower().split()

#Eliminar las palabras vacías (stop words)
stop_words = set(stopwords.words('english'))
filtered_words = [word for word in all_words if word not in stop_words]

#Crear una cadena de texto a partir de las palabras filtradas
text = ' '.join(filtered_words)

#Crear la nube de palabras
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

#Visualizar la nube de palabras
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Nube de Palabras: Palabras más Frecuentes en los Diálogos de Friends')
plt.axis('off')
plt.show()


La nube de palabras muestra las más frecuentes en los diálogos de la serie. Las palabras más grandes son las más frecuentes, mientras que las pequeñas son menos frecuentes.


*   Temas principales: aparecen todos los nombres de los protagonistas. El predominante sería Ross y le seguirían Monica, Joey, Phoebe Chandler y Rachel.
*   Palabras comunes como yeah, oh, okay, know, well, guy, sorry, oh god... son muy habituales. Son palabras comunes en el lenguaje coloquial y sirven para expresar emociones, confirmar entendimiento o facilitar una conversación fluida. Se puede atribuir a la serie un tono coloquial, uso frecuente del lenguaje sencillo y cercano y la exploración de temas comunes y cotidianos relacionado con la vida de jóvenes adultos en Nueva York.  



Longitud de los diálogos por temporada

In [ ]:
#Calcular la longitud de los diálogos por temporada
df_personajes_principales['Dialogue_Length'] = df_personajes_principales['Text'].apply(lambda x: len(x.split()))

#Calcular la longitud promedio de los diálogos por temporada
avg_dialogue_length_season = df_personajes_principales.groupby('Season')['Dialogue_Length'].mean()

#Visualizar la longitud promedio de los diálogos por temporada
plt.figure(figsize=(10, 6))
avg_dialogue_length_season.plot(kind='bar', color='lightseagreen')
plt.title('Longitud Promedio de los Diálogos por Temporada')
plt.xlabel('Temporada')
plt.ylabel('Longitud Promedio de los Diálogos')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


La longitud promedio de los diálogos por temporada se mantiene bastante estable a lo largo de las temporadas. Destacan ligeramente por encima del resto en longitud las temporadas 6 y 9. Ambas temporadas tienen momentos importantes para la serie como es la pedida de matrimonio entre Monica y Chandler (temporada 6), mientras que la novena temporada es la penúltima de la serie y ocurren muchas tramas entre todos los personajes.

**Análisis de la frecuencia de los personajes en cada temporada**

In [ ]:
#Lista de personajes principales
personajes_principales = ['joey', 'monica', 'rachel', 'phoebe', 'ross', 'chandler']

#Filtrar el DataFrame para incluir solo los personajes principales
df_personajes_principales = df[df['Speaker'].str.lower().isin(personajes_principales)]

#Contar la frecuencia de cada personaje en cada temporada
character_counts = df_personajes_principales.groupby(['Season', 'Speaker']).size().unstack(fill_value=0)

#Visualizar los resultados en un gráfico de barras
plt.figure(figsize=(12, 6))
character_counts.plot(kind='bar', stacked=True)
plt.title('Frecuencia de Personajes Principales por Temporada')
plt.xlabel('Temporada')
plt.ylabel('Frecuencia')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Personaje')
plt.tight_layout()
plt.show()

Observamos que todos los personajes principales tienen una presencia constante a lo largo de las temporadas, por lo que se demuestra el papel relevante y recurrente de todos en la serie. Sin embargo, hay variaciones en la frecuencia en diferentes temporadas.

*   Ross y Rachel destacan y su aparición en la serie es ligeramente más alta que la de otros personajes principales en varias temporadas. Esto puede reflejar su relación y la trama asociada en la serie.
*   Aun así, se obseva un equilibrio entre los personajes y esto sugiere que Friends se centra en la interacción entre las historias de todos los protagonistas de manera equitativa.



Relaciones entre los personajes

Vamos a crear un gráfico que visualice las relaciones entre los protagonistas para tener una visión más completa sobre la dinámica entre los personajes.

In [ ]:
#Crear una lista de protagonistas
protagonists_list = df_personajes_principales['Speaker'].unique()

#Crear un DataFrame para almacenar las interacciones de cada protagonista con los demás
interactions_with_others = pd.DataFrame(index=protagonists_list, columns=protagonists_list)

#Calcular las interacciones de cada protagonista con los demás
for protagonist in protagonists_list:
    # Filtrar las interacciones del protagonista actual con los demás
    interactions_with_others[protagonist] = df_personajes_principales[df_personajes_principales['Speaker'] != protagonist]['Speaker'].value_counts()

#Visualizar las interacciones de cada protagonista con los demás como un gráfico de barras
plt.figure(figsize=(12, 8))
interactions_with_others.plot(kind='bar', stacked=True, cmap='Paired')
plt.title('Frecuencia de Interacciones entre Protagonistas en Toda la Serie')
plt.xlabel('Protagonista')
plt.ylabel('Frecuencia de Interacciones')
plt.xticks(rotation=45)
plt.legend(title='Protagonista')
plt.tight_layout()
plt.show()



En este gráfico se pueden apreciar los personajes que interactúan más con otros. Por ejemplo, Ross y Rachel.

**PREPROCESAMIENTO DE LOS DATOS**

Vamos a proceder a la tokenización, eliminación de caracteres especiales y de stopwords, lematización y conversión a minúsculas.

In [ ]:
#Inicializar lematizador
lemmatizer = WordNetLemmatizer()

#Función para eliminar números
def remove_numbers(text):
    return ''.join([i for i in text if not i.isdigit()])

#Tokenización
df['Text'] = df['Text'].apply(word_tokenize)

#Conversión a minúsculas
df['Text'] = df['Text'].apply(lambda x: [word.lower() for word in x])

#Eliminación de caracteres no deseados y números
punctuation = set(string.punctuation)
df['Text'] = df['Text'].apply(lambda x: [remove_numbers(word) for word in x if word not in punctuation])

#Eliminación de stopwords
stop_words = set(stopwords.words('english'))
df['Text'] = df['Text'].apply(lambda x: [word for word in x if word not in stop_words])

#Lematización
df['Text'] = df['Text'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

#Unir las palabras nuevamente en una sola cadena
df['Text'] = df['Text'].apply(lambda x: ' '.join(x))


**ANÁLISIS DE SENTIMIENTO**

Vamos a contrastar diferentes diccionarios de sentimientos y evaluaremos cómo se diferencian sus puntajes. Utilizaremos Vader y TextBlob para asignar puntajes de sentimiento a algunas palabras de ejemplo.

In [ ]:
#Inicializar Vader
sid = SentimentIntensityAnalyzer()

#Palabras de ejemplo
example_words = ["happy", "sad", "excited", "angry"]

#Puntajes de sentimiento en Vader
vader_scores = {word: sid.polarity_scores(word)['compound'] for word in example_words}

print("Palabra\t\tVader")
print("--------------------")
for word in example_words:
    print(f"{word}\t\t{vader_scores[word]}")


In [ ]:
#Palabras de ejemplo
example_words = ["happy", "sad", "excited", "angry"]

#Puntajes de sentimiento en TextBlob
textblob_scores = {word: TextBlob(word).sentiment.polarity for word in example_words}

print("\nPalabra\t\tTextBlob")
print("--------------------")
for word in example_words:
    print(f"{word}\t\t{textblob_scores[word]}")


Vemos que hay diferencias en los puntajes de sentimiento asignados por Vader y TextBlob en las mismas palabras.


*   'Happy': ambos diccionarios consideran que es una palabra positiva, pero TextBlob le asigna una puntuación mucho más alta.
*   Para el resto de palabras ambos diccionarios asignan puntajes bastante similares.



Comenzamos con el análisis de sentimiento de los textos de Friends. Seguiremos utilizando ambos diccionarios para contrastar resultados

In [ ]:
#Inicializar Vader
sid = SentimentIntensityAnalyzer()

#Aplicar Vader a los diálogos de Friends
df['Sentiment'] = df['Text'].apply(lambda x: sid.polarity_scores(x)['compound'])

#Crear una función para asignar la categoría de sentimiento
def categorize_sentiment(score):
    if score > 0:
        return 'positivo'
    elif score < 0:
        return 'negativo'
    else:
        return 'neutro'
#Aplicar la función a los puntajes de sentimiento compuesto
df['Sentiment'] = df['Sentiment'].apply(categorize_sentiment)

from textblob import TextBlob
# Definir una función para asignar la categoría de sentimiento en TextBlob
def categorize_sentiment_textblob(score):
    if score > 0:
        return 'positivo'
    elif score < 0:
        return 'negativo'
    else:
        return 'neutro'
#Aplicar TextBlob a los diálogos de Friends
df['Sentiment_TextBlob'] = df['Text'].apply(lambda x: TextBlob(x).sentiment.polarity)
# Aplicar la función de categorización a los puntajes de sentimiento en TextBlob
df['Sentiment_TextBlob'] = df['Sentiment_TextBlob'].apply(categorize_sentiment_textblob)


Visualizamos los resultados obtenidos con ambos diccionarios:

In [ ]:
#Configurar el tamaño de la figura
plt.figure(figsize=(12, 6))

#Histograma de los puntajes de sentimiento obtenidos con Vader
plt.subplot(1, 2, 1)
plt.hist(df['Sentiment'], bins=20, color='skyblue', edgecolor='black')
plt.title('Distribución de Sentimientos (Vader)')
plt.xlabel('Puntaje de Sentimiento')
plt.ylabel('Frecuencia')

#Histograma de los puntajes de sentimiento obtenidos con TextBlob
plt.subplot(1, 2, 2)
plt.hist(df['Sentiment_TextBlob'], bins=20, color='salmon', edgecolor='black')
plt.title('Distribución de Sentimientos (TextBlob)')
plt.xlabel('Puntaje de Sentimiento')
plt.ylabel('Frecuencia')

#Mostrar los gráficos
plt.tight_layout()
plt.show()


Ambos histogramas muestran la variedad de sentimientos en los diálgos de Friends. La tendencia principal la marca la neutralidad o la positividad en la mayoría de los casos. Vader tiende a marcar puntajes más positivos que TextBlob.

**Análisis de sentimientos por personaje y temporada**

In [ ]:
#Lista de protagonistas principales
protagonistas = ['joey', 'monica', 'rachel', 'phoebe', 'ross', 'chandler']

#Filtrar el DataFrame para incluir solo los diálogos de los protagonistas principales
df_protagonistas = df[df['Speaker'].str.lower().isin(protagonistas)]

#Aplicar TextBlob a los diálogos de Friends de los protagonistas principales
df_protagonistas['Sentiment_TextBlob'] = df_protagonistas['Text'].apply(lambda x: TextBlob(x).sentiment.polarity)

#Agrupar los puntajes de sentimiento de TextBlob por personaje y episodio y calcular los promedios
textblob_grouped = df_protagonistas.groupby(['Speaker', 'Season'])['Sentiment_TextBlob'].mean().reset_index()

#Crear un gráfico de barras para cada protagonista principal
for personaje in protagonistas:
    data_personaje = textblob_grouped[textblob_grouped['Speaker'] == personaje]
    plt.figure(figsize=(12, 6))
    plt.bar(data_personaje['Season'], data_personaje['Sentiment_TextBlob'])
    plt.title(f'Puntaje Promedio de Sentimiento por Temporada ({personaje.capitalize()})')
    plt.xlabel('Temporada')
    plt.ylabel('Puntaje Promedio de Sentimiento')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.show()


In [ ]:
#Definir colores para cada personaje
colors = {'joey': 'black', 'phoebe': 'red', 'chandler': 'orange', 'monica': 'blue', 'ross': 'brown', 'rachel': 'green'}

#Crear el gráfico de dispersión con colores personalizados
plt.figure(figsize=(12, 8))
for personaje in protagonistas:
    data_personaje = textblob_grouped[textblob_grouped['Speaker'] == personaje]
    plt.scatter(data_personaje['Season'], data_personaje['Sentiment_TextBlob'], label=personaje, color=colors.get(personaje, 'blue'))
plt.title('Puntaje Promedio de Sentimiento por Temporada (Gráfico de Dispersión)')
plt.xlabel('Temporada')
plt.ylabel('Puntaje Promedio de Sentimiento')
plt.legend(title='Personaje')
plt.tight_layout()
plt.show()

Los puntajes de sentimiento más altos se pueden ver representados en el personaje de Phoebe, en rojo en el gráfico, especialmente en las temporadas 4, 2 y 7. El mayor puntaje de sentimiento se produce en la temporada 4, en la que Phoebe se convierte en madre y gana mayor protagonismo en la trama. Por otro lado, Mónica tiene el puntaje más bajo en la temporada 1, mientras que Joey y Chandler también presentan puntajes bajos de sentimiento a lo largo de la serie.

In [ ]:
#Agrupar los datos por personaje y calcular el promedio del sentimiento positivo
average_positive_sentiment = df_protagonistas[df_protagonistas['Sentiment_TextBlob'] > 0].groupby('Speaker')['Sentiment_TextBlob'].mean()

#Encontrar el personaje con el mayor y el menor sentimiento positivo
max_positive_sentiment_character = average_positive_sentiment.idxmax()
min_positive_sentiment_character = average_positive_sentiment.idxmin()

#Mostrar los resultados
print("Personaje con mayor sentimiento positivo:", max_positive_sentiment_character)
print("Personaje con menor sentimiento positivo:", min_positive_sentiment_character)


El personaje con mayor sentimiento positivo a lo largo de todas las temporadas de Friends es Phoebe, mientras que Joey es el personaje que menor sentimiento positivo presenta.

**ANÁLISIS DE TÓPICOS:**

Vamos a llevar a cabo el modelado de LDA para identificar los temas principales en los diálogos de Friends.
*   Creamos un diccionario de palabras a través de los textos preprocesados.
*   Convertimos el corpus de texto en formato que pueda utilizar LDA.
*   Entrenamos un modelo LDA con cinco tópicos.
*   Imprimimos los tópicos y las palabras más relevantes en cada tópico.





In [ ]:
#Crear un diccionario de palabras a partir de los textos preprocesados de los protagonistas
dictionary = corpora.Dictionary(df_protagonistas['Text'].apply(lambda x: x.split()))

#Convertir el corpus en un formato que pueda ser utilizado por el modelo LDA
corpus = [dictionary.doc2bow(text.split()) for text in df_protagonistas['Text']]

#Entrenar el modelo LDA
lda_model = gensim.models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, random_state=42)

#Imprimir los tópicos y las palabras más relevantes en cada tópico
for topic_id, topic_words in lda_model.print_topics():
    print(f"Topic {topic_id}: {topic_words}")



Los resultados muestran los cinco tems identificados por el modelo LDA junto a sus palabras más relevantes.

*   Tópico 0: Expresiones y verbos comunes. Expresiones y palabras comunes que se utilizan con frecuencia en los diálogos.
*   Tópico 1: Expresiones de emoción y afirmación. Oh, yeah, god, thank.
*   Tópico 2: Saludos y nombres de personajes. Hi, wait, y nombres de los personajes.
*   Tópico 3: Expresiones y verbos comunes. Contracciones de verbos en inglés y expresiones comunes.
*   Tópico 4: Puntuación y verbos comunes. Hey, got, yes, get.








In [ ]:
#Crear word clouds para cada tópico
for topic_id, topic_words in lda_model.print_topics():
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(topic_words)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(f'Word Cloud - Tópico {topic_id}')
    plt.axis('off')
    plt.show()

#Crear gráficos de barras para la distribución de palabras clave en cada tópico
for topic_id, topic_words in lda_model.print_topics():
    words = [word.split("*")[1].replace('"', '').strip() for word in topic_words.split(" + ")]
    prob = [float(word.split("*")[0]) for word in topic_words.split(" + ")]
    plt.figure(figsize=(10, 5))
    plt.bar(words, prob, color='skyblue')
    plt.title(f'Distribución de palabras clave - Tópico {topic_id}')
    plt.xlabel('Palabra')
    plt.ylabel('Probabilidad')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


MODELIZACIÓN:

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import gensim
from gensim import matutils

#Asignación de tópicos a documentos
topics_distribution = [lda_model.get_document_topics(doc) for doc in corpus]

# Crear una matriz de características 'X' utilizando la asignación de tópicos a documentos como características
topics_distribution = [lda_model.get_document_topics(doc) for doc in corpus]
X = matutils.corpus2dense(topics_distribution, num_terms=lda_model.num_topics).T

#Asegurar que X y y tengan el mismo número de muestras
y = df['Sentiment']
y = y[:X.shape[0]]

#Dividir los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Inicializar y entrenar un modelo de clasificación (por ejemplo, regresión logística)
model = LogisticRegression()
model.fit(X_train, y_train)

#Predecir sobre el conjunto de prueba
y_pred = model.predict(X_test)

#Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Probamos con otros modelos en un intento de mejorar los resultados:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

#Inicializar modelos de clasificación
models = [
    LogisticRegression(),
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    SVC()
]

#Nombres de los modelos para identificarlos en la salida
model_names = [
    "Regresión Logística",
    "K Vecinos Más Cercanos",
    "Árbol de Decisión",
    "Bosque Aleatorio",
    "SVM"
]

#Iterar sobre los modelos y entrenarlos
for model, name in zip(models, model_names):
    #Entrenar el modelo
    model.fit(X_train, y_train)

    #Predecir sobre el conjunto de prueba
    y_pred = model.predict(X_test)

    #Calcular la precisión
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{name} - Accuracy: {accuracy}")




Los mejores resultados los obtenemos con el modelo de SVC, Regresión Logística y Random Forest.

Probamos ahora con los puntajes obtenidos con TextBlob:

In [ ]:
#Utilizar los puntajes de sentimiento obtenidos por TextBlob
y_textblob = df['Sentiment_TextBlob']
y_textblob = y_textblob[:X.shape[0]]  # Asegurar que X y y_textblob tengan el mismo número de muestras

#Dividir los datos en conjunto de entrenamiento y conjunto de prueba
X_train_tb, X_test_tb, y_train_tb, y_test_tb = train_test_split(X, y_textblob, test_size=0.2, random_state=42)

#Inicializar modelos de clasificación
models = [
    LogisticRegression(),
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    SVC()
]

#Nombres de los modelos para identificarlos en la salida
model_names = [
    "Regresión Logística",
    "K Vecinos Más Cercanos",
    "Árbol de Decisión",
    "Bosque Aleatorio",
    "SVM"
]

#Iterar sobre los modelos y entrenarlos con los puntajes de TextBlob
for model, name in zip(models, model_names):
    #Entrenar el modelo
    model.fit(X_train_tb, y_train_tb)

    #Predecir sobre el conjunto de prueba
    y_pred_tb = model.predict(X_test_tb)

    #Calcular la precisión
    accuracy_tb = accuracy_score(y_test_tb, y_pred_tb)
    print(f"{name} - Accuracy (TextBlob): {accuracy_tb}")


Vemos que los resultados obtenidos con TextBlob son ligeramente superiores a los obtenidos con Vader. Los modelos que mejor funcionan son los de Regresión Logística y SVM, por encima del 0.55 en ambos casos.

Representación gráfica de los resultados obtenidos por Vader y Textblob:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

#Paleta de colores pastel
colors_pastel = sns.color_palette("pastel")

#Precisión de los modelos con Vader
accuracies_vader = [0.43024218588283997, 0.4055061063961913, 0.40478161871248186, 0.4254812668184641, 0.4339681225419168]

#Precisión de los modelos con TextBlob
accuracies_textblob = [0.5555785551645622, 0.4830262885530946, 0.440695508176361, 0.49751604222728213, 0.5555785551645622]

#Nombres de los modelos
model_names = [
    "Regresión Logística",
    "K Vecinos Más Cercanos",
    "Árbol de Decisión",
    "Bosque Aleatorio",
    "SVM"
]

#Crear el gráfico de barras horizontal con la paleta de colores pastel
plt.figure(figsize=(10, 6))
plt.barh(model_names, accuracies_vader, color=colors_pastel, label='Vader')
plt.barh(model_names, accuracies_textblob, color=colors_pastel, alpha=0.5, label='TextBlob')
plt.xlabel('Accuracy')
plt.title('Comparación de Precisión de Modelos con Vader y TextBlob')
plt.legend()
plt.gca().invert_yaxis()  # Invertir el eje y para que el modelo superior esté en la parte superior
plt.show()



Los modelos de Regresión Logística y SVM muestran una precisión considerablemente mayor en comparación con otros modelos, tanto para los puntajes de sentimiento obtenidos por Vader como por TextBlob. Son los modelos más efectivos para clasificar los sentimientos expresados en los diálogos de Friends.
Los modelos entrenados con los puntajes de TextBlob mostraron una precisión generalmente más alta. Sin embargo, incluso los mejores modelos tienen una precisión que va desde el 40 al 55%, lo que indica que aún hay margen de mejora en la capacidad de estos modelos para capturar la complejidad de los sentimientos expresados en los diálogos de la serie.

Optimización de parámetros y validación cruzada:

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

#Definir los hiperparámetros a ajustar
param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf', 'poly', 'sigmoid']}

#Inicializar el modelo SVM
svm = SVC()

#Realizar la búsqueda de hiperparámetros con validación cruzada
svm_grid = GridSearchCV(svm, param_grid, cv=5, scoring='accuracy')
svm_grid.fit(X_train, y_train)

#Obtener los mejores hiperparámetros y la precisión del modelo
best_params_svm = svm_grid.best_params_
best_score_svm = svm_grid.best_score_

print("Mejores hiperparámetros para SVM:", best_params_svm)
print("Precisión del modelo SVM:", best_score_svm)

